<a href="https://colab.research.google.com/github/Je73hwg/AI-LAb/blob/main/resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
class Literal:
    def __init__(self, predicate, negated=False):
        self.predicate = predicate
        self.negated = negated

    def __str__(self):
        return f"\u00ac{self.predicate}" if self.negated else self.predicate

    def __eq__(self, other):
        return isinstance(other, Literal) and self.predicate == other.predicate and self.negated == other.negated

    def __hash__(self):
        return hash((self.predicate, self.negated))

    def negate(self):
        return Literal(self.predicate, not self.negated)


class Clause:
    def __init__(self, literals):
        self.literals = set(literals)

    def __str__(self):
        if not self.literals:
            return "False"
        return " ∨ ".join(str(literal) for literal in self.literals)

    def __eq__(self, other):
        return isinstance(other, Clause) and self.literals == other.literals

    def __hash__(self):
        return hash(frozenset(self.literals))


class Resolution:
    def __init__(self):
        self.clauses = []
        self.resolution_tree = []
        self.resolved_clauses = set()

    def add_clause(self, clause):
        if clause not in self.clauses:
            self.clauses.append(clause)

    def resolve(self, clause1, clause2):
        for literal1 in clause1.literals:
            for literal2 in clause2.literals:
                if literal1.predicate == literal2.predicate and literal1.negated != literal2.negated:
                    # Remove complementary literals and merge the remaining ones
                    new_literals = (clause1.literals | clause2.literals) - {literal1, literal2}
                    return Clause(new_literals)
        return None

    def prove(self, goal):
        print("Starting proof...")
        negated_goal = Clause([goal.negate()])
        self.add_clause(negated_goal)
        new_clauses = set()

        while True:
            print("Resolving clauses...")
            pairs = [(self.clauses[i], self.clauses[j]) for i in range(len(self.clauses)) for j in range(i + 1, len(self.clauses))]

            for clause1, clause2 in pairs:
                if (clause1, clause2) in self.resolved_clauses or (clause2, clause1) in self.resolved_clauses:
                    continue
                self.resolved_clauses.add((clause1, clause2))

                resolvent = self.resolve(clause1, clause2)
                if resolvent:
                    print(f"Resolvent: {resolvent}")
                    self.resolution_tree.append((clause1, clause2, resolvent))

                    if not resolvent.literals:
                        print("Contradiction found!")
                        return True
                    new_clauses.add(resolvent)

            if not new_clauses:
                print("No new clauses, proof failed.")
                return False

            for clause in new_clauses:
                self.add_clause(clause)
            new_clauses = set()

    def print_resolution_tree(self):
        print("Resolution Tree:")
        for clause1, clause2, resolvent in self.resolution_tree:
            print(f"{clause1} + {clause2} -> {resolvent}")


# Define predicates and clauses
leela_mother_oshin = Literal("mother(leela, oshin)")
leela_alive = Literal("alive(leela)")
leela_parent_oshin = Literal("parent(leela, oshin)")
leela_older_oshin = Literal("older(leela, oshin)")

# Convert rules to CNF
clause1 = Clause([leela_mother_oshin])  # mother(leela, oshin)
clause2 = Clause([leela_alive])  # alive(leela)
clause3 = Clause([Literal("mother(leela, oshin)").negate(), leela_parent_oshin])  # ¬mother(leela, oshin) ∨ parent(leela, oshin)
clause4 = Clause([leela_parent_oshin.negate(), leela_alive.negate(), leela_older_oshin])  # ¬parent(leela, oshin) ∨ ¬alive(leela) ∨ older(leela, oshin)

# Create resolution instance
resolution = Resolution()
resolution.add_clause(clause1)
resolution.add_clause(clause2)
resolution.add_clause(clause3)
resolution.add_clause(clause4)

# Prove goal
goal = leela_older_oshin
result = resolution.prove(goal)
resolution.print_resolution_tree()
print("Proved" if result else "Failed to prove")


Starting proof...
Resolving clauses...
Resolvent: parent(leela, oshin)
Resolvent: ¬parent(leela, oshin) ∨ older(leela, oshin)
Resolvent: ¬alive(leela) ∨ ¬mother(leela, oshin) ∨ older(leela, oshin)
Resolvent: ¬alive(leela) ∨ ¬parent(leela, oshin)
Resolving clauses...
Resolvent: ¬alive(leela) ∨ older(leela, oshin)
Resolvent: ¬mother(leela, oshin) ∨ older(leela, oshin)
Resolvent: ¬parent(leela, oshin)
Resolvent: ¬mother(leela, oshin) ∨ older(leela, oshin)
Resolvent: ¬alive(leela) ∨ ¬mother(leela, oshin)
Resolvent: ¬alive(leela) ∨ older(leela, oshin)
Resolvent: ¬parent(leela, oshin)
Resolvent: ¬alive(leela) ∨ ¬mother(leela, oshin)
Resolvent: older(leela, oshin)
Resolvent: ¬alive(leela)
Resolving clauses...
Resolvent: ¬alive(leela)
Resolvent: older(leela, oshin)
Resolvent: ¬mother(leela, oshin)
Resolvent: False
Contradiction found!
Resolution Tree:
mother(leela, oshin) + parent(leela, oshin) ∨ ¬mother(leela, oshin) -> parent(leela, oshin)
alive(leela) + ¬alive(leela) ∨ ¬parent(leela, oshin)